In [113]:
import src.config as config

Earliest date: 2022-01-01 00:00:08


Latest date: 2023-12-31 23:59:59


Number of rows: 77964130

In [115]:
from datetime import datetime, timedelta,timezone
import pandas as pd

# Data in backfill feature store goes from 2022-01-01 00:00:00+00:00 to '2024-11-30 23:00:00+0000', tz='UTC')

# current_date = pd.to_datetime('2024-11-30 00:00:00+0000').floor('h')
from datetime import datetime, timezone
current_date = datetime.now(timezone.utc)

#current_date = pd.to_datetime(aware_utcnow()).floor('h')
print(f'{current_date=}')
print(type(current_date),f': {current_date=}')

current_date=datetime.datetime(2025, 1, 28, 0, 9, 54, 849644, tzinfo=datetime.timezone.utc)
<class 'datetime.datetime'> : current_date=datetime.datetime(2025, 1, 28, 0, 9, 54, 849644, tzinfo=datetime.timezone.utc)


In [117]:
# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=60)
fetch_data_from

# data from 2023-11-05 23:00:00 to '2023-12-03 23:00:00' 
# to fill the 28 days backfill 

datetime.datetime(2024, 11, 29, 0, 9, 54, 849644, tzinfo=datetime.timezone.utc)

In [118]:
# Problem arising with data dates here 

In [119]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    from_date_ = pd.Timestamp(from_date - timedelta(days=7*52)).replace(tzinfo=None)
    to_date_ = pd.Timestamp(to_date - timedelta(days=7*52)).replace(tzinfo=None)

    print(f'{from_date_=}, {to_date_=}')

    # Load data and normalize timezones for pickup_datetime
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'], errors='coerce').dt.tz_localize(None)
    rides = rides[rides.pickup_datetime >= from_date_]

    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2['pickup_datetime'] = pd.to_datetime(rides_2['pickup_datetime'], errors='coerce').dt.tz_localize(None)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # Shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [120]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)
rides

from_date_=Timestamp('2023-12-01 00:09:54.849644'), to_date_=Timestamp('2024-01-30 00:09:54.849644')
File 2023-12 was already in local storage
File 2024-01 was already in local storage


,pickup_datetime,pickup_location_id
7553,2024-11-29 03:21:20,1
18143,2024-11-29 07:56:50,1
18672,2024-11-29 09:17:27,1
30960,2024-11-29 11:55:46,1
40173,2024-11-29 11:59:57,1
...,...,...
2625232,2025-01-27 22:23:10,265
2625233,2025-01-27 22:23:10,265
2623446,2025-01-27 22:30:37,265
2627683,2025-01-27 23:04:45,265


In [129]:
print(rides['pickup_datetime'].min())
print(rides['pickup_datetime'].max())

2024-11-29 00:09:57
2025-01-28 00:09:54


In [130]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 262/262 [00:00<00:00, 993.72it/s] 


In [131]:
print(ts_data['pickup_hour'].min())
print(ts_data['pickup_hour'].max())

2024-11-29 00:00:00
2025-01-28 00:00:00


In [132]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

ts_data

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2024-11-29 00:00:00+00:00,10,4,1732838400000
1,2024-11-29 01:00:00+00:00,7,4,1732842000000
2,2024-11-29 02:00:00+00:00,10,4,1732845600000
3,2024-11-29 03:00:00+00:00,4,4,1732849200000
4,2024-11-29 04:00:00+00:00,0,4,1732852800000
...,...,...,...,...
377537,2025-01-27 20:00:00+00:00,0,199,1738008000000
377538,2025-01-27 21:00:00+00:00,0,199,1738011600000
377539,2025-01-27 22:00:00+00:00,0,199,1738015200000
377540,2025-01-27 23:00:00+00:00,0,199,1738018800000


In [125]:
print(ts_data['pickup_hour'].min())
print(ts_data['pickup_hour'].max())

2024-11-29 00:00:00
2025-01-28 00:00:00


In [133]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,# "time_series_hourly_feature_group"
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

2025-01-28 00:20:16,376 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-28 00:20:16,390 INFO: Initializing external client
2025-01-28 00:20:16,390 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-28 00:20:17,406 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


In [134]:
print(f"Total rows in ts_data: {len(ts_data)}")

Total rows in ts_data: 377542


In [135]:
# This is only the last 28 days from the current date set at the top btw,  notebook 11 has taken all of the 2022-2024
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 377542/377542 | Elapsed Time: 01:06 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1192104/jobs/named/time_series_hourly_feature_group_3_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_3_offline_fg_materialization', 'SPARK'),
 None)